# Introduction

julie.tsitron@parks.nyc.gov 1/8/2020

This notebook is for testing various ArcGIS API for Python functions that allow for pushing/updating GIS data to production servers. Ultimately, the script from this notebook can be used as a template for the technical implementation (i.e., pushing clean data into production ESRI-based geodatabases) of Structures, Sites and Units, CPAs, and other Agency Data Model data products.

# Imports and Connections to DBs

In [1]:
# import urllib3
# from urllib.parse import quote
# from urllib.parse import quote_plus

import numpy as np
import json
import sqlalchemy
import os
import shapely
import pytz
from datetime import datetime
import pandas as pd
import pyodbc
import sys
sys.path.append('../') 
from IPM_Shared_Code.Python.geo_functions import read_geosql
from IPM_Shared_Code.Python.utils import get_config
from IPM_Shared_Code.Python.email_functions import get_contacts, read_template, send_email

from arcgis.gis import GIS
from arcgis.features import GeoAccessor, GeoSeriesAccessor, SpatialDataFrame, FeatureLayer, FeatureSet

In [2]:
config = get_config('c:\Projects\config.ini')

driver = config['srv']['driver']
server = config['srv']['server']
parksgis = config['srv']['parksgis']
data_parks_server = config['srv']['data_parks']
structuresdb = config['db']['structuresdb']
portal = config['url']['portal']
structures_url = config['url']['structures']

In [3]:
con = pyodbc.connect('Driver={' + driver + '};Server=' + server +
                     ';Database=' + structuresdb + ';Trusted_Connection=Yes;')
con_gis = pyodbc.connect('Driver={SQL Server};Server=' + parksgis +
                         ';Database=;Trusted_Connection=Yes;')

In [6]:
crsr = con.cursor()
crsr.execute("{CALL structuresdb.dbo.sp_i_tbl_overlap_exceptions}")
crsr.commit()
crsr.close()

In [ ]:
crsr = con.cursor()
crsr.execute("{CALL structuresdb.dbo.sp_i_tbl_audit_structures}")
crsr.commit()
crsr.close()

In [ ]:
crsr = con.cursor()
sql_stmt = '''
exec [structuresdb].[dbo].[sp_i_tbl_delta_structures_archive] 
'''
crsr.execute("{CALL [structuresdb].[dbo].[sp_i_tbl_delta_structures_archive] }")
crsr.commit()
crsr.close()

In [ ]:
# params = urllib.parse.quote_plus(r'Driver=' + driver + ';Server=' + server +
#                                  ';Database=' + structuresdb +
#                                  ';Trusted_Connection=Yes;')
# engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)
# connection = engine.connect()

# Delta Table from structuresdb

In [57]:
sql_structures = '''
select * from [ParksGIS].[DPR].[STRUCTURE_EVW] 
'''
structures_latest = read_geosql(sql_structures,
                                con_gis,
                                geom_raw='SHAPE',
                                geom_col='geometry')

In [39]:
def make_tz_aware_UTC(row, column):
    return pd.to_datetime(row[column]).tz_localize('UTC').astimezone(pytz.UTC)

In [40]:
def make_tz_aware_EST(row, column):
    return pd.to_datetime(row[column]).tz_localize('EST').astimezone(pytz.UTC)

In [58]:
structures_latest['last_edited_date'] = structures_latest.apply(
    lambda row: make_tz_aware_UTC(row, 'last_edited_date'), axis=1)

In [92]:
sql_str_deltas = '''
select * FROM [structuresdb].[dbo].[tbl_delta_structures] 
'''
struct_deltas = read_geosql(sql_str_deltas,
                            con,
                            geom_raw='shape',
                            geom_col='geometry')

In [93]:
struct_deltas.rename(columns={
    'objectid': 'OBJECTID',
    'parks_id': 'SYSTEM',
    'bin': 'BIN',
    'bbl': 'BBL',
    'doitt_id': 'DOITT_ID',
    'ground_elevation': 'Ground_Elevation',
    'height_roof': 'Height_Roof',
    'alteration_year': 'Alteration_Year',
    'construction_year': 'Construction_Year',
    'demolition_year': 'Demolition_Year'
},
                     inplace=True)

In [94]:
struct_deltas.head()

,fid,OBJECTID,SYSTEM,BIN,BBL,DOITT_ID,Ground_Elevation,Height_Roof,Construction_Year,Alteration_Year,Demolition_Year,api_call,doitt_source,date_stamp,geometry


In [95]:
if not struct_deltas.empty:
    struct_deltas['date_stamp'] = struct_deltas.apply(
        lambda row: make_tz_aware_UTC(row, 'date_stamp'), axis=1)

In [60]:
last_push_api = structures_latest[
    (structures_latest['last_edited_user'] == 'NYCDPR\py_services')
    & (structures_latest['last_edited_date'] == structures_latest[
        (structures_latest['last_edited_user'] == 'NYCDPR\py_services'
         )]['last_edited_date'].max())].reset_index()['last_edited_date'].loc[0]
last_push_api

Timestamp('2020-03-09 15:07:14+0000', tz='UTC')

In [91]:
struct_deltas[struct_deltas['date_stamp']>last_push_api]

,fid,OBJECTID,SYSTEM,BIN,BBL,DOITT_ID,Ground_Elevation,Height_Roof,Construction_Year,Alteration_Year,Demolition_Year,api_call,doitt_source,date_stamp,geometry


In [29]:
struct_deltas = struct_deltas[struct_deltas['date_stamp']>last_push_api]

In [30]:
struct_deltas.head()

,fid,OBJECTID,SYSTEM,BIN,BBL,DOITT_ID,Ground_Elevation,Height_Roof,Construction_Year,Alteration_Year,Demolition_Year,api_call,doitt_source,date_stamp,geometry
4161,4162,12136,R088-BLG5143,5143399,5009390120,954559,147.0,19.5,1901.0,NaN,None,U,current,2020-03-07 03:00:09.297000+00:00,"POLYGON ((950839.2118229717 150902.7225227952,..."
4162,4163,4987,M160-BLG0679,1060055,1020080029,1283720,15.0,NaN,NaN,NaN,None,U,current,2020-03-07 03:00:09.297000+00:00,"POLYGON ((1001132.62435931 237049.2743107229, ..."
4163,4164,5125,X141-BLG0941,2128987,2048390020,1283727,155.0,NaN,NaN,NaN,None,U,current,2020-03-07 03:00:09.297000+00:00,"POLYGON ((1023291.007863984 262709.2856546342,..."
4164,4165,5624,M070-BLG5624,1090506,1014540001,1283745,13.0,NaN,NaN,NaN,None,U,current,2020-03-07 03:00:09.297000+00:00,"POLYGON ((995163.7084756494 215865.263857305, ..."
4165,4166,6205,Q009-ZN02-BLG0409,4095654,4039140001,1283770,10.0,NaN,NaN,NaN,None,U,current,2020-03-07 03:00:09.297000+00:00,"POLYGON ((1025401.079103142 228393.2122265548,..."


In [ ]:
# def to_wkt(row):
#     return row.wkt

# Write delta table to geojson file, create FeatureSet, connect to service, push edits, if any. 
# Otherwise, just connect to Structures feature layer and query:

In [33]:
if not struct_deltas.empty:
    ## WRITE TO GEOJSON
    today = datetime.now().strftime('%Y%m%d')
    snapshot = r'C:\\Projects\\AgencyDataModel\\Develop\\Structures\\delta_snapshots/' + today
    if os.path.exists(snapshot):
        struct_deltas.to_file(snapshot + '/deltas.geojson',
                              encoding='utf-8',
                              driver='GeoJSON')
    else:
        os.makedirs(snapshot)
        struct_deltas.to_file(snapshot + '/deltas.geojson',
                              encoding='utf-8',
                              driver='GeoJSON')

    ## READ GEOJSON FILE TO (GEO)JSON OBJECT
    with open(snapshot + '/deltas.geojson') as data:
        geojson_deltas = json.load(data)

    ## CREATE ARCGIS FEATURESET FROM GEOJSON OBJECT
    fromJSON_deltas = FeatureSet.from_geojson(geojson_deltas)

    ## CONNECT TO PUBLISHED DATASET VIA GIS OBJECT
    gis = GIS(url=portal)

    ## CONNECT TO FEATURE LAYER DIRECTLY AND QUERY
    strct_lyr_url = structures_url

    lyr_structures = FeatureLayer(strct_lyr_url)
    structures_features = lyr_structures.query()

    ## MAKE EDITS
    try:
        update_result = lyr_structures.edit_features(
            updates=fromJSON_deltas.features)
    except:
        error_mssg = 'Updates were not pushed successfully to Structures. Please see Notebook output/logs for further details.'
        send_email('mycontacts.txt',
                   'fail_update_mssg.txt',
                   subject='structures push: FAILED!',
                   e=error_mssg)
    else:
        num_updates = len(fromJSON_deltas.features)
        success_mssg = 'There were ' + str(num_updates) + ' updates pushed to Production.'
        send_email('mycontacts.txt',
                   'success_structures_mssg.txt',
                   subject='structures push: SUCCESS!',
                   e=success_mssg)
else:
    mssg = 'There were no updates to push to the Structure layer.'
    send_email('mycontacts.txt','fail_update_mssg.txt', subject = 'structures: NO UPDATES to push', e=mssg)

C:\ProgramData\Anaconda3\lib\site-packages\arcgis\features\feature.py:793: UserWarning: crs has been deprecated and will be ignored. Please see: https://tools.ietf.org/html/rfc7946#section-4 for more information.
  " see: https://tools.ietf.org/html/rfc7946#section-4 for" +\


NameError: name 'num_updates' is not defined